Created by 642115004 Kitdanai Palee

In [9]:
import os
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from ordered_set import OrderedSet
import re
from sklearn.feature_extraction.text import CountVectorizer
import itertools
from string import ascii_lowercase
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

IULA

In [10]:
topdir = '../Resource/iula/'
all_content = []
for dirpath, dirnames, filename in os.walk(topdir):
    for name in filename:
        if name.endswith('plain.txt'):
            with open(os.path.join(dirpath, name)) as f:
                all_content.append(f.read())

Error: `beter`

Candidates: bitter, better, batter, butter, otter

In [45]:
COCA = pd.DataFrame([['bitter',16976], ['better',495426], ['otter', 1388], ['batter',5932], ['butter',28812]], columns=['word','frequency'])
COCA_pop = 1001610938
COCA['P(w)'] = COCA['frequency']/COCA_pop
COCA['rank'] = COCA['frequency'].rank(ascending=False, method='min').astype(int)
COCA

,word,frequency,P(w),rank
0,bitter,16976,0.000017,3
1,better,495426,0.000495,1
2,otter,1388,0.000001,5
3,batter,5932,0.000006,4
4,butter,28812,0.000029,2


In [46]:
def create_stem_cache(cleaned_description):
    tokenized_description = [word_tokenize(s) for s in cleaned_description]
    concated = np.unique(np.concatenate(tokenized_description))
    stem_cache = {}
    ps = PorterStemmer()
    for s in concated:
        stem_cache[s] = ps.stem(s)
    return stem_cache

def create_custom_preprocessor(stop_dict, stem_cache):
    def custom_preprocessor(s):
        ps = PorterStemmer()
        s = re.sub(r'[^A-Za-z]', ' ', s)
        s = re.sub(r'\s+', ' ', s)
        s = word_tokenize(s)
        s = list(OrderedSet(s) - stop_dict)
        s = [word for word in s if len(word)>2]
        s = [stem_cache[w] if w in stem_cache else ps.stem(w) for w in s]
        s = ' '.join(s)
        return s
    return custom_preprocessor


In [47]:
stem_cache = create_stem_cache(all_content)
stop_dict = set(stopwords.words('English'))
preProcess = create_custom_preprocessor(stop_dict,stem_cache)

In [48]:
processed_content = [preProcess(s) for s in all_content]

In [49]:
vectorizer = CountVectorizer()
vectorizer.fit(processed_content)
freq_iula = vectorizer.transform(processed_content)
freq_iula = pd.DataFrame(freq_iula.todense(), columns=vectorizer.get_feature_names_out()).sum()
freq_iula

aaa             1
aaaaaa          1
aalborg         2
aarhu           1
aaron           3
               ..
zuckerkandl     1
zurich          4
zvi             1
zygos           1
zygot          10
Length: 22529, dtype: int64

In [50]:
query = ['bitter', 'better', 'otter', 'batter', 'butter']
transformed_query = [vectorizer.inverse_transform(vectorizer.transform([q])) for q in query]
query_freq = pd.Series([freq_iula.T.loc[tq[0]].values[0] if len(tq[0]) > 0 else 0 for tq in transformed_query], index= query)
query_freq

bitter     1
better    84
otter      0
batter     1
butter     1
dtype: int64

In [51]:
IULA = pd.DataFrame(query_freq, columns=['frequency'])
IULA_pop = len(freq_iula)
IULA['P(w)'] = IULA['frequency']/IULA_pop
IULA['rank'] = IULA['frequency'].rank(ascending=False).astype(int)
IULA

,frequency,P(w),rank
bitter,1,0.000044,3
better,84,0.003729,1
otter,0,0.000000,5
batter,1,0.000044,3
butter,1,0.000044,3


In [52]:
norvig = pd.read_csv('http://norvig.com/ngrams/count_1edit.txt', sep='\t', encoding="ISO-8859-1", header=None)
norvig.columns = ['term', 'edit']
norvig = norvig.set_index('term')
print(norvig.head())


      edit
term      
e|i    917
a|e    856
i|e    771
e|a    749
a|i    559


In [53]:
norvig_orig = pd.read_csv('http://norvig.com/ngrams/count_big.txt', sep='\t', encoding="ISO-8859-1", header=None)
norvig_orig = norvig_orig.dropna()
norvig_orig.columns=['term','freq']
print(norvig_orig.head())


    term   freq
0      a  21160
1    aah      1
2  aaron      5
3     ab      2
4  aback      3


In [54]:
def get_count(c,norvig_orig):
    return norvig_orig.apply(lambda x: x.term.count(c) * x.freq, axis=1).sum()


In [55]:
character_set = list(map(''.join, itertools.product(ascii_lowercase, repeat=1))) + list(map(''.join, itertools.product(ascii_lowercase, repeat=2)))

pool = ThreadPool(8) #8 is your #compute cores
freq_list = pool.starmap(get_count, zip(character_set, itertools.repeat(norvig_orig)))

freq_df = pd.DataFrame([character_set, freq_list], index=['char', 'freq']).T
freq_df = freq_df.set_index('char')


Error: `btter`




| Candidate | Correct Letter | Error Letter |
|------------|----------------|--------------|
| Bitter     | i              | -            |
| Better     | e              | -            |
| otter      | o              | -            |
| Batter     | a              | -            |
| Butter     | u              | -            |


In [57]:
COCA['P(x|w)'] = [
    (norvig.loc['b|bi'].values / freq_df.loc['bi'].values)[0],  # Bitter
    (norvig.loc['b|be'].values / freq_df.loc['be'].values)[0],  # Better
    (norvig.loc['b|o'].values / freq_df.loc['o'].values)[0],  # Butler
    (norvig.loc['b|ba'].values / freq_df.loc['ba'].values)[0],  # Batter
    (norvig.loc['b|bu'].values / freq_df.loc['bu'].values)[0]  # Butter
]
COCA

,word,frequency,P(w),rank,P(x|w)
0,bitter,16976,0.000017,3,0.008083
1,better,495426,0.000495,1,0.001941
2,otter,1388,0.000001,5,0.000003
3,batter,5932,0.000006,4,0.000702
4,butter,28812,0.000029,2,0.002545


In [58]:
COCA['109 P(x|w)P(w)'] = 1e9 * COCA['P(w)'] * COCA['P(x|w)']
COCA

,word,frequency,P(w),rank,P(x|w),109 P(x|w)P(w)
0,bitter,16976,0.000017,3,0.008083,136.998703
1,better,495426,0.000495,1,0.001941,959.936664
2,otter,1388,0.000001,5,0.000003,0.003584
3,batter,5932,0.000006,4,0.000702,4.159030
4,butter,28812,0.000029,2,0.002545,73.220471
